In [1]:
from dotenv import load_dotenv
import google.generativeai as genai

import os
from src.services.llm_generation.llm import Prompt, Model
from src.services.tools.News.news import API_KEY
from src.services.tools.agent_tools import AgentTools
from openai import OpenAI

load_dotenv()
openai_api_key = os.getenv("openai_key")
gemini_api_key = os.getenv("gemini_key")
claude_api_key = os.getenv("claude_key")
tools_path = os.getenv("tools")

gemini_model="gemini-2.5-flash"
llm_model=gemini_model
gemini_key=gemini_api_key
api_key = gemini_key
gemini_base_url= "https://generativelanguage.googleapis.com/v1beta"






model=OpenAI(api_key=gemini_key,base_url=gemini_base_url)
tools_openai, tools_gemini = AgentTools().list_of_tools(tools_path)
role="user"
context=""
source=""
message ="quelle est la capitale de la Belgique?"
prompt=Prompt(role,context,source,message)

C:\Users\tallar\Documents\PROJETS\GenAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

print(tools_gemini)
 # récupère la clé depuis GEMINI_API_KEY ou GOOGLE_API_KEY


[{'function_declarations': [{'name': 'list_of_tools', 'description': 'No description available.', 'parameters': {'type': 'object', 'properties': {'self': {'type': 'string', 'description': 'Paramètre self'}, 'racine': {'type': 'string', 'description': 'Paramètre racine'}}, 'required': ['self', 'racine']}}]}, {'function_declarations': [{'name': 'f_get_bank_transaction', 'description': 'Permet de récupérer la liste des transactions bancaires\nReturns:\n     str: la liste des transactions bancaires.', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}]}, {'function_declarations': [{'name': 'f_get_data', 'description': ':param id: identifiant du redevable\n:return: la liste des créances ou des transactions encore ourvertes', 'parameters': {'type': 'object', 'properties': {'id': {'type': 'string', 'description': 'Paramètre id'}}, 'required': ['id']}}]}, {'function_declarations': [{'name': 'f_get_rag_response', 'description': 'à définir.\n\nArgs:\n    query (str): à définir\n

In [3]:

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash", # Replace with your actual key
    tools=tools_gemini
)
# Generate content


def gemini_process(model, agent, message):
    response = model.generate_content(message)
    tool_responses=[]
    # Print the result
    for part in response.candidates[0].content.parts:
        # Vérifier si la partie est un appel de fonction
        if hasattr(part, 'function_call'):
            tool_call = part.function_call

            # Obtenir le nom de la fonction et ses arguments
            tool_name = tool_call.name
            tool_args = tool_call.args

            print(f"Le modèle a demandé d'exécuter la fonction : {tool_name}")
            print(f"Avec les arguments : {tool_args}")
            result = agent.use_tool(tool_name,tool_args)

            tool_responses.append(result)

    if tool_responses:
        print("\n-------------------------------------------------------------")
        print("Résultat des outils obtenu. On le renvoie au modèle...")
        print("-------------------------------------------------------------")

        # 1. Créez un objet de contenu pour le message initial de l'utilisateur
        user_content = genai.protos.Content(
            parts=[genai.protos.Part(text=message)],
            role='user'
        )

        # 2. L'historique de la conversation commence par le message de l'utilisateur...
        chat_history = [user_content]

        # 3. ...suivi par la réponse du modèle qui contient l'appel de fonction...
        chat_history.append(response.candidates[0].content)

        # 4. ...suivi par la réponse de l'outil qui contient les résultats.
        tool_content = genai.protos.Content(
            parts=[genai.protos.Part(text=tool_responses[0])],
            role='user'
        )
        chat_history.append(tool_content)

        # Relancez generate_content avec l'historique complet pour obtenir la réponse finale
        final_response = model.generate_content(chat_history)
        print("\n-------------------------------------------------------------")
        print("Réponse finale du modèle :")
        print("-------------------------------------------------------------")
        return final_response.text
    else:
        print("\n-------------------------------------------------------------")
        print("Le modèle a fourni une réponse textuelle directe :")
        print("-------------------------------------------------------------")
        return response.text


In [4]:
from src.services.Agent.agent import Agent
from src.services.llm_generation.llm import Model
from src.services.tools.agent_tools import AgentTools
from src.services.memory.agent_memory import AgentMemory

In [5]:

model=Model("openai","gpt-4o")
memoire = AgentMemory("poc","test_memoire.txt")
agent=Agent(model=model,tools=AgentTools(),memory=memoire)


Fichier 'C:\Users\tallar\Documents\PROJETS\GenAI\src\services\memory\Files\poc-test_memoire.txt' créé avec succès.


In [6]:

agent.run("user","","peux tu me communiquer la liste de mes dernieres transactions bancaires?")


appel de fonction 1
ChatCompletionMessageToolCall(id='call_AZad1JGE5EQdcMRbzanHRuP5', function=Function(arguments='{}', name='f_get_bank_transaction'), type='function')
call_id: call_AZad1JGE5EQdcMRbzanHRuP5
name: f_get_bank_transaction
arguments: {}
----le prompt après appels des fonctions----
[ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AZad1JGE5EQdcMRbzanHRuP5', function=Function(arguments='{}', name='f_get_bank_transaction'), type='function')]), {'role': 'tool', 'tool_call_id': 'call_AZad1JGE5EQdcMRbzanHRuP5', 'content': '"01/08/2025 Salaire \\u2013 Entreprise XYZ +2500,00 \\u20ac Solde : 2500,00 \\u20ac\\n        03/08/2025 Paiement carte \\u2013 Supermarch\\u00e9 -75,60 \\u20ac Solde : 2424,40 \\u20ac\\n        05/08/2025 Virement re\\u00e7u \\u2013 Amis +150,00 \\u20ac Solde : 2574,40 \\u20ac\\n        07/08/2025 Pr\\u00e9l\\u00e8vement automatique \\u2013 \

'Here is the list of recent bank transactions:\n\n1. **01/08/2025**: Salary from Company XYZ, **+2500.00 €**. Balance: **2500.00 €**\n2. **03/08/2025**: Card payment at Supermarket, **-75.60 €**. Balance: **2424.40 €**\n3. **05/08/2025**: Received transfer from a friend, **+150.00 €**. Balance: **2574.40 €**\n4. **07/08/2025**: Automatic debit for Electricity, **-120.45 €**. Balance: **2453.95 €**\n5. **10/08/2025**: ATM withdrawal, **-200.00 €**. Balance: **2253.95 €**\n6. **12/08/2025**: Card payment at Restaurant, **-45.80 €**. Balance: **2208.15 €**\n7. **15/08/2025**: Transfer to savings account, **-500.00 €**. Balance: **1708.15 €**\n8. **18/08/2025**: Card payment at Online Store, **-89.99 €**. Balance: **1618.16 €**\n9. **21/08/2025**: Automatic debit for Internet, **-35.00 €**. Balance: **1583.16 €**\n10. **23/08/2025**: Personal loan repayment, **+300.00 €**. Balance: **1883.16 €**'

In [7]:

memoire.save_memory()
memoire.load_memory()
print(memoire.get_memories())

["# Mémoire de l'agent IA", '[2025-09-02 18:28:50] question : peux tu me communiquer la liste de mes dernieres transactions bancaires? ; réponse : Here is the list of recent bank transactions:  1. **01/08/2025**: Salary from Company XYZ, **+2500.00 €**. Balance: **2500.00 €** 2. **03/08/2025**: Card payment at Supermarket, **-75.60 €**. Balance: **2424.40 €** 3. **05/08/2025**: Received transfer from a friend, **+150.00 €**. Balance: **2574.40 €** 4. **07/08/2025**: Automatic debit for Electricity, **-120.45 €**. Balance: **2453.95 €** 5. **10/08/2025**: ATM withdrawal, **-200.00 €**. Balance: **2253.95 €** 6. **12/08/2025**: Card payment at Restaurant, **-45.80 €**. Balance: **2208.15 €** 7. **15/08/2025**: Transfer to savings account, **-500.00 €**. Balance: **1708.15 €** 8. **18/08/2025**: Card payment at Online Store, **-89.99 €**. Balance: **1618.16 €** 9. **21/08/2025**: Automatic debit for Internet, **-35.00 €**. Balance: **1583.16 €** 10. **23/08/2025**: Personal loan repayment